# 2D NSE — Parameter Sweep (Classical vs NMSI–π*–γ_diss–e)

Grid search over ν, Aπ, ζ, λ_e, α_e. Saves CSV/NPZ diagnostics for open peer review.

In [1]:
import numpy as np, itertools as it, csv
from pathlib import Path
import matplotlib.pyplot as plt

from math import pi

# (Reuse minimal 2D solver pieces from the compare notebook)
N = 64; L = 2*np.pi
x = np.linspace(0, L, N, endpoint=False); y = np.linspace(0, L, N, endpoint=False)
X, Y = np.meshgrid(x, y, indexing='ij')
kx = 1j*2*np.pi*np.fft.fftfreq(N, d=L/(2*np.pi*N))
ky = 1j*2*np.pi*np.fft.fftfreq(N, d=L/(2*np.pi*N))
KX, KY = np.meshgrid(kx, ky, indexing='ij')
k2 = KX**2 + KY**2; k2[0,0]=1.0

def project(u_hat, v_hat):
    div_hat = KX*u_hat + KY*v_hat
    return u_hat - KX*div_hat/k2, v_hat - KY*div_hat/k2
def grad(h):
    return KX*h, KY*h
def nonlinear(u,v):
    u_hat, v_hat = np.fft.fft2(u), np.fft.fft2(v)
    ux, uy = [np.fft.ifft2(g).real for g in grad(u_hat)]
    vx, vy = [np.fft.ifft2(g).real for g in grad(v_hat)]
    Nu = -(u*ux + v*uy); Nv = -(u*vx + v*vy)
    return project(np.fft.fft2(Nu), np.fft.fft2(Nv))
def vorticity(u,v):
    u_hat, v_hat = np.fft.fft2(u), np.fft.fft2(v)
    uy = np.fft.ifft2(grad(u_hat)[1]).real
    vx = np.fft.ifft2(grad(v_hat)[0]).real
    return vx - uy
def energy(u,v):
    return 0.5*np.mean(u*u+v*v)
def enstrophy(u,v):
    w = vorticity(u,v)
    return np.mean(w*w)
def init_TG(A=1.0,k=1):
    u0 =  A*np.sin(k*X)*np.cos(k*Y); v0 = -A*np.cos(k*X)*np.sin(k*Y)
    u0_hat, v0_hat = project(np.fft.fft2(u0), np.fft.fft2(v0))
    return np.fft.ifft2(u0_hat).real, np.fft.ifft2(v0_hat).real
def step(u,v,rhs,dt):
    # SSPRK3
    ru1, rv1 = [np.fft.ifft2(h).real for h in rhs(u,v)]
    u1, v1 = u+dt*ru1, v+dt*rv1
    ru2, rv2 = [np.fft.ifft2(h).real for h in rhs(u1,v1)]
    u2 = 0.75*u + 0.25*(u1 + dt*ru2)
    v2 = 0.75*v + 0.25*(v1 + dt*rv2)
    ru3, rv3 = [np.fft.ifft2(h).real for h in rhs(u2,v2)]
    un = (1/3)*u + (2/3)*(u2 + dt*ru3)
    vn = (1/3)*v + (2/3)*(v2 + dt*rv3)
    return un, vn
def make_rhs(nu=8e-4, A_pi=0.0, omega_pi=3.5, zeta=0.0, z_thresh=np.inf,
             lam_e=0.0, alpha_e=0.0):
    def rhs(u,v,t=[0.0]):
        t0 = t[0]
        Nu, Nv = nonlinear(u,v)
        u_hat, v_hat = np.fft.fft2(u), np.fft.fft2(v)
        visc_u, visc_v = -nu*k2*u_hat, -nu*k2*v_hat
        aug = 0.0
        if A_pi!=0.0: aug += A_pi*np.sin(omega_pi*t0)
        if zeta!=0.0 and enstrophy(u,v)>z_thresh: aug += -zeta
        if lam_e!=0.0: aug += -lam_e*np.exp(-alpha_e*t0)
        Ru, Rv = Nu+visc_u+aug*u_hat, Nv+visc_v+aug*v_hat
        return project(Ru,Rv)
    return rhs
def run_case(T=4.0, dt=2e-3, nu=8e-4, augmented=False,
             A_pi=0.15, omega_pi=3.5, zeta=0.8, z_thresh=0.02,
             lam_e=0.6, alpha_e=0.25, A0=1.0, k0=1):
    u,v = init_TG(A=A0,k=k0)
    t=0.0; E_hist=[]; Z_hist=[]; T_hist=[]
    nsteps=int(T/dt)
    for n in range(nsteps):
        rhs = make_rhs(nu=nu,
                       A_pi=(A_pi if augmented else 0.0), omega_pi=omega_pi,
                       zeta=(zeta if augmented else 0.0), z_thresh=z_thresh,
                       lam_e=(lam_e if augmented else 0.0), alpha_e=alpha_e)
        rhs.__defaults__[0][0] = t
        u,v = step(u,v,rhs,dt); t+=dt
        if n%10==0:
            E_hist.append(energy(u,v)); Z_hist.append(enstrophy(u,v)); T_hist.append(t)
    return np.array(T_hist), np.array(E_hist), np.array(Z_hist)

In [2]:
# Parameter grid
nus      = [6e-4, 8e-4]
lam_es   = [0.3, 0.6]
A_pis    = [0.10, 0.15]
zetas    = [0.6, 0.8]
alpha_es = [0.2, 0.3]

outdir = Path('sweep_out'); outdir.mkdir(exist_ok=True)
rows = []
for nu, lam_e, A_pi, zeta, alpha_e in it.product(nus, lam_es, A_pis, zetas, alpha_es):
    T1,E1,Z1 = run_case(nu=nu, augmented=False)
    T2,E2,Z2 = run_case(nu=nu, augmented=True, A_pi=A_pi, zeta=zeta, lam_e=lam_e, alpha_e=alpha_e)
    rows.append({
        'nu':nu,'lam_e':lam_e,'A_pi':A_pi,'zeta':zeta,'alpha_e':alpha_e,
        'E_final_classical':float(E1[-1]), 'E_final_aug':float(E2[-1]),
        'Z_max_classical':float(Z1.max()), 'Z_max_aug':float(Z2.max())
    })
    # save per-case npz
    tag=f"nu{nu}_le{lam_e}_Ap{A_pi}_ze{zeta}_ae{alpha_e}".replace('.','p')
    np.savez(outdir/f'case_{tag}.npz', T1=T1,E1=E1,Z1=Z1,T2=T2,E2=E2,Z2=Z2)

with open(outdir/'summary.csv','w',newline='') as f:
    w=csv.DictWriter(f, fieldnames=list(rows[0].keys()))
    w.writeheader(); w.writerows(rows)
print('Saved sweep to', outdir)